# Markov decision process(MDP)

에이전트에게 완전한 문제를 제시하는 환경을 마르코프 결정 과정(Markov decision process, MDP)라고 한다. <br>
이러한 환경은 

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt

C:\Users\a\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 오픈AI에서 특정 환경을 불러오는 코드
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [29]:
gamma = 0.99

def discount_rewards(r):
    # 보상의 1D 실수 배열을 취해서 할인된 보상을 계산
    discounted_r = np.zeros_like(r)
    running_add = 0
    
    for t in range(r.size-1, -1, -1):
        running_add = running_add*gamma+r[t]
        discounted_r[t] = running_add
    
    return discounted_r

class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # 네트워크의 피드포워드 부분, 에이전트는 상태를 받아서 액션을 출력
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        # 히든레이어 설정
        hidden = slim.fully_connected(inputs=self.state_in, num_outputs=h_size, activation_fn=tf.nn.relu, biases_initializer=None)
        # 출력레이어 설정
        self.output = slim.fully_connected(inputs=hidden, num_outputs=a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        # 출력레이어에서 값이 가장 큰 값의 인덱스 선택
        self.chosen_action = tf.aregmax(input=self.output, 1)
        
        # 학습 과정 구현, 비용을 계산하기 위해 봇아과 액션을 네트워크에 피드하고, 이를 통해 네트워크 업데이트
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        